In [ ]:
#데이터셋 다운로드 링크: https://www.kaggle.com/yelp-dataset/yelp-dataset/data?select=yelp_academic_dataset_review.json 
                        #yelp_academic_dataset_review.json(5.89 GB) → 이 파일 다운로드 
#추가 참고자료: https://wikidocs.net/book/2155

# Chapter 3. Text Data: Flattening, Filtering, and Chunking
텍스트 데이터의 기본적인 feature engineering을 다루는 챕터<br>
텍스트 데이터를 수치화, 전처리, 다양한 단위(word, <i>n</i>-gram, phrase)로 분리하는 방법에 대해서 다룸

## 1. Bag-of-X: Turning Natural Text into Flat Vectors 
토큰의 출현 빈도를 통해 텍스트 데이터를 수치화하는 방법<br>
토큰(token): 텍스트 처리에 있어 유용하고 의미있는 단위 (보통 단어)

### 1. Bag-of-Words
- 텍스트 데이터를 각 단어의 카운트로 이루어진 flat vector로 변환 (기존 텍스트의 문장구조가 사라지기 때문에 'flat')
- feautre는 단어, feature vector은 그 단어의 카운트를 나타냄

<img src="img/figure3-1.png" width="500" height="500">
<center>(Feature Engineering for Machine Learning by Alice Zheng and Amanda Casari (O’Reilly))</center>  

##### 시각화
- data vector를 feature space에 나타낼 수 있고, 반대로 feature vector를 data space에 나타낼 수도 있음

<img src="img/figure3-2.png" width="500" height="500">
<center>data vector를 feature space에 나타냄</center>
<img src="img/figure3-3.png" width="500" height="500">
<center>feature vecter를 data space에 나타냄</center>
<center>(Feature Engineering for Machine Learning by Alice Zheng and Amanda Casari (O’Reilly))</center> 

##### 문제점
- 단어 간 순서에 대한 정보가 없음
- 단어 간 위계에 대한 정보를 표현할 수 없음 (예: 동물, 강아지, 고양이가 모두 동일한 위계를 가짐)
- semantic meaning을 없애버리게 됨 (예: "not bad"이 긍정의 의미를 가지고 있지만, 단일 단어로 나누게 되면 이런 의미를 상실하게 됨)

### 2. Bag-of-<i>n</i>-Grams 
- bag-of-words의 개념을 확장시킨 방법
- <i>n</i>-gram: a sequence of <i>n</i> tokens
- <i>n</i>-grams는 기존 단어 순서에 대한 정보를 어느정도 가지고 있음

##### 문제점
- 보통 <i>n</i>-grams은 단일 단어보다 그 갯수가 많음. 그로 인해 <i>n</i>-grams를 모델에 이용할 때 더 많은 비용(storage cost, computation cost)이 들고, 데이터들이 더 sparse 해짐(데이터는 그대로인데 feature dimension이 커지기 때문) 
- 따라서 <i>n</i>이 증가한다고 해서 모델의 정확도가 항상 향상되는 것이 아님. bigram 또는 trigram을 가장 많이 사용

##### <i>n</i>-grams 구하기 
- CountVectorizer transformer 이용

In [2]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer 

In [4]:
#https://github.com/alicezheng/feature-engineering-book/blob/master/03.01_Count_words.ipynb 참고 

#10,000개의 리뷰 데이터를 불러와서 데이터프레임 만들기
f = open('C:\\Users\\이인주\\Desktop\\2020 summer study\\feature_engineering\\data\\yelp_academic_dataset_review.json', 'r', encoding='UTF-8') 
        #데이터 경로 입력
js = [] #리뷰 데이터를 추가할 리스트 생성 

for i in range(10000):
    js.append(json.loads(f.readline())) #json.loads(): json 형식의 문자열을 파이썬 사전의 형태로 변환
                                        #readline(): 한줄씩 읽어들임 
f.close()

review_df = pd.DataFrame(js) #리스트를 데이터프레임으로 변환
review_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,-MhfebM0QIsKt87iDN-FNw,0,2015-04-15 05:21:16,0,xQY8N_XvtGbearJ5X4QryQ,2.0,"As someone who has worked with many museums, I...",5,OwjRMXRC0KyPrIlcjaXeFQ
1,lbrU8StCq3yDfr-QMnGrmQ,0,2013-12-07 03:16:52,1,UmFMZ8PyXZTY2QcwzsfQYA,1.0,I am actually horrified this place is still in...,1,nIJD_7ZXHq-FX8byPMOkMQ
2,HQl28KMwrEKHqhFrrDqVNQ,0,2015-12-05 03:18:11,0,LG2ZaYiOgpr2DK_90pYjNw,5.0,I love Deagan's. I do. I really do. The atmosp...,1,V34qejxNsCbcgD8C0HVk-Q
3,5JxlZaqCnk1MnbgRirs40Q,0,2011-05-27 05:30:52,0,i6g_oA9Yf9Y31qt0wibXpw,1.0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",0,ofKDkJKXSKZXu5xJNGiiBQ
4,IS4cv902ykd8wj1TR0N3-A,0,2017-01-14 21:56:57,0,6TdNDKywdbjoTkizeMce8A,4.0,"Oh happy day, finally have a Canes near my cas...",0,UgMW8bLE0QMJDCkQ1Ax5Mg


In [6]:
#리뷰 데이터 예시 
review_df['text'][0]

'As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It\'s what real estate agents would call "cozy" or "charming" - basically any euphemism for small.\n\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\n\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\n* it\'s not kid friendly at all. Seriously, don\'t bring them.\n* the security is not trained properly for the show. When the curating and design teams collaborate for exhibitions, there is a definite flow. That means visitors should view the ar

In [7]:
# https://github.com/alicezheng/feature-engineering-book/blob/master/03.01_Count_words.ipynb 참고

#텍스트를 unigram, bigram, trigram으로 나누어주는 각 feature transformer 설정 
unigram_converter = CountVectorizer(token_pattern='(?u)\\b\\w+\\b') #token_pattern='(?u)\\b\\w+\\b': 정규식 표현. 공백 사이에 있는 2글자 이상의 문자열을 찾아 토큰으로 분해 
bigram_converter = CountVectorizer(ngram_range=(2,2), token_pattern='(?u)\\b\\w+\\b') #ngram_range(min_n, max_n): n-grams의 n의 최소값과 최대값 
trigram_converter = CountVectorizer(ngram_range=(3,3), token_pattern='(?u)\\b\\w+\\b')

In [23]:
# https://github.com/alicezheng/feature-engineering-book/blob/master/03.01_Count_words.ipynb 참고

unigram = unigram_converter.fit(review_df['text']) #앞에서 만든 feature transformer를 통해 텍스트를 unigram으로 나누기
one = unigram.get_feature_names() #get_feature_names(): feature의 목록을 볼 수 있음 
one[1000:1020]

['advised',
 'adviser',
 'advises',
 'advising',
 'advisor',
 'advisors',
 'advocate',
 'aerobic',
 'aerobics',
 'aesthetic',
 'aesthetically',
 'aesthetician',
 'aesthetics',
 'af',
 'afaik',
 'afb',
 'affair',
 'affect',
 'affected',
 'affecting']

In [25]:
# https://github.com/alicezheng/feature-engineering-book/blob/master/03.01_Count_words.ipynb 참고

bigram = bigram_converter.fit(review_df['text']) 
two = bigram.get_feature_names() 
two[2000:2020]

['2 pounds',
 '2 pours',
 '2 price',
 '2 priced',
 '2 prime',
 '2 probably',
 '2 puke',
 '2 pumps',
 '2 queen',
 '2 queens',
 '2 quotes',
 '2 r',
 '2 race',
 '2 races',
 '2 read',
 '2 reasons',
 '2 recessed',
 '2 refills',
 '2 registers',
 '2 regular']

In [26]:
# https://github.com/alicezheng/feature-engineering-book/blob/master/03.01_Count_words.ipynb 참고

trigram = trigram_converter.fit(review_df['text']) 
third = trigram.get_feature_names() 
third[2000:2010]

['13 seconds to',
 '13 that included',
 '13 the 12oz',
 '13 through monday',
 '13 tip the',
 '13 tip to',
 '13 unfortunately they',
 '13 was definitely',
 '13 were scratched',
 '13 when making']

## 2. Filtering for Cleaner Features 
텍스트 데이터를 전처리하는 방법에 대해 다룸

### 1. Stopwords(불용어)
- 분석에 있어 크게 도움이 되지 않는 토큰을 제거 
- 파이썬 NLP 패키지 중 하나인 'NLTK'에서 다양한 언어의 불용어 목록을 제공함 
- English stopword list: a, about, above, am, an, been, didn't ...

### 2. Frequency-Based Filtering 
- 토큰의 <b>출현 빈도</b>에 기반하여 데이터를 전처리 

#### (1) Frequent words 
- 빈도수가 큰 토큰을 제거
- 출현 빈도수를 이용하면 general-purpose stopword 뿐만 아니라 corpus-specific common word까지 필터링할 수 있음<br>
예시: New Youk Times 기사 텍스트에서 'New York Times'라는 구(phrase)와 각 단어의 출현 빈도수는 큼. 그러나 해당 구와 단어가 텍스트 데이터에서 중요한 의미를 가지고 있는 것은 아님
- 출현 빈도수에 기반하여 필터링 할 때 빈번함 정도의 cutoff를 정하는 것이 어려움
- 보통 frequency-based filtering과 stopword list를 함께 사용하여 전처리 함

#### (2) Rare words 
- 빈도수가 작은(1-2회) 토큰들을 제거
- 보통 rare word는 predictor로서 unreliable하고 많은 computational 비용을 만들어내기 때문에 제거함
- rare word를 벡터 리스트에서 지워버리는 방법과 'garbage'라는 bin을 통해 rare word의 갯수를 따로 세는 방법이 있음 

<img src="img/figure3-4.png" width="500" height="500">
<center>(Feature Engineering for Machine Learning by Alice Zheng and Amanda Casari (O’Reilly))</center> 

### 3. Stemming and Lemmatization
텍스트 데이터의 단어들을 단순하게 파싱(parsing)하면 한 단어의 다양한 형태가 모두 서로 다른 독립적인 단어로 카운트되는 문제점이 발생<br>
(예: '꽃', '꽃들'이 결국 같은 단어임에도 서로 다른 독립적인 단어로 카운트)
<br><br>
이를 해결하기 위해 <b>Stemming(어간추출)</b> 또는 <b>Lemmatization(표제어추출)</b>을 사용함

#### 3.1 Stemming (어간추출) 
- 어간(stem)을 추출하는 작업 (어간: 단어의 핵심적인 의미를 담고 있는 부분. (예: 꽃들 → 꽃))
- 형태학적 분석을 단순화함. 정해진 규칙을 보고 단어의 어미를 자르는 어림짐작의 작업 
- 섬세한 작업이 아니기 때문에 어간 추출 후에 나오는 결과가 사전에 존재하지 않은 단어일 수 있음 
- 품사정보가 보존되지 않음 (POS(Part-Of-Speech) 태그를 고려하지 않음)
- Porter stemmer: 영어에 사용할 수 있는 stemming tool

In [1]:
import nltk
text = "As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas"
tokenizer = nltk.tokenize.TreebankWordTokenizer() #.TreebankWordTokenizer(): 문법규칙에 기반하여 텍스트의 토큰을 구분
tokens = tokenizer.tokenize(text)#텍스트 데이터를 토큰화함
print(tokens)

['As', 'someone', 'who', 'has', 'worked', 'with', 'many', 'museums', ',', 'I', 'was', 'eager', 'to', 'visit', 'this', 'gallery', 'on', 'my', 'most', 'recent', 'trip', 'to', 'Las', 'Vegas']


In [28]:
L = []
stemmer = nltk.stem.PorterStemmer() #stemmer 설정

for token in tokens:
    stem = stemmer.stem(token)
    L.append(stem)
    
print(L)

['As', 'someon', 'who', 'ha', 'work', 'with', 'mani', 'museum', ',', 'I', 'wa', 'eager', 'to', 'visit', 'thi', 'galleri', 'on', 'my', 'most', 'recent', 'trip', 'to', 'la', 'vega']


#### 3.2. Lemmatization (표제어추출)
- 표제어(Lemma)를 추출하는 작업 (표제어: 기본 사전형 단어)
- 단어의 접사(affix)와 어간(stem)을 분리하는 형태학적 파싱을 통해 표제어를 추출
- 품사정보가 보존(POS 태그를 보존)

In [13]:
nltk.download('wordnet') #.WordNetLemmatizer() 실행시 오류가 떴을 때

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\이인주\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [31]:
L = []
lemmatizer = nltk.stem.WordNetLemmatizer() #lemmatizer 설정

for token in tokens:
    lemma = lemmatizer.lemmatize(token)
    L.append(lemma)
    
print(L)

['As', 'someone', 'who', 'ha', 'worked', 'with', 'many', 'museum', ',', 'I', 'wa', 'eager', 'to', 'visit', 'this', 'gallery', 'on', 'my', 'most', 'recent', 'trip', 'to', 'Las', 'Vegas']


- 표제어 추출기(lemmatizer)에 본래 단어의 품사 정보를 입력해줄 경우 더 정확하게 표제어를 출력할 수 있음

In [32]:
lemmatizer.lemmatize('has', 'v')

'have'

## 3. Atoms of Meaning: From Words to <i>n</i>-Grams to Phrases 
텍스트 데이터를 word, <i>n</i>-grams, phrase의 형태로 분리하는 방법에 대하여 다룸<br>
텍스트를 <i>n</i>-grams, phrase의 형태로 분리하면 텍스트 문장구조에 대한 정보를 얻을 수 있음 

### 1. Parsing and Tokenization
텍스트 데이터를 토큰의 형태로 분리하는 과정 
##### Parsing
문서가 순수 텍스트 이외의 다른 요소들을 포함하고 있을 때 사용<br>
예: 다루고자 하는 텍스트 데이터가 이메일의 형태라면 From, To 등은 별개로 다루어야 함 
##### Tokenization
파싱을 통해 얻은 순수 텍스트 데이터를 토큰의 형태로 분리<br>

### 2. Collocation Extracion for Phrase Detection 
우리는 텍스트를 읽을 때, 단어 하나하나를 이해하기 보단 구(phrase) 자체를 이해함으로써 텍스트 내용을 이해하는 경우가 많음<br>
텍스트 데이터에서 개별적인 단어가 아닌 collocation을 추출해내는 방법을 다룸 

##### Collocation
- 특정 의미를 나타내기 위해 흔히 함께 쓰이는 2개 이상의 단어들의 결합 
- collocation은 각 단어 의미의 합 이상의 의미를 지님 (예: strong tea > strong + tea )
- 반드시 문장 안에서 연이어 위치할 필요는 없음 (예: Emma knocked on the door → knock door)
- Bag-of-<i>n</i>-grams 방법으로는 텍스트로부터 collocation을 제대로 추출해내기 어려움 

#### Collocation을 추출해내는 방법
##### 1. Predefining 
- 자주 사용하는 collocation들을 사전에 정의해두는 방법 
- 신조어 등을 바로 반영하지 못하는 문제점이 있음 
- 따라서 통계적인 방법을 사용하는 것이 좋음 (Frequency-based methods, Hypothesis testing, Chunking and part-of-speech tagging)

##### 2. Frequency-based methods 
- 빈번하게 나타난 <i>n</i>-grams을 이용하여 collocation을 추출
- 문제점: 빈도수가 크다고 해서 반드시 의미가 있는 <i>n</i>-grams인 것은 아님 

##### 3. Hypothesis testing for collocation extraction
- 두 단어가 같이 사용되는 확률이 우연 수준보다 큰지 확인 
- 대표적인 방법: likelihood ratio test

##### likelihood ratio test 

<img src="img/figure3-5.png" width="500" height="500">
<img src="img/figure3-6.png" width="300" height="300">

- likelihood function L(Data; H): 해당 가설이 참일 때 데이터셋에서 단어 빈도수가 나타날 확률을 도출하는 함수   
- 두 단어가 같이 사용되는 확률이 우연 수준의 확률보다 클 때 likelihood ratio 값(log λ)이 작아짐
- likelihood ratio 값(log λ)이 작은 단어의 쌍을 feature로 선택 

##### 4. Chunking and Part-Of-Speech(POS; 품사) tagging
- 긴 collocation을 추출해내는 방법  
- 품사 정보를 기반으로 sequences of tokens를 만들어냄  
- 텍스트를 토큰화하는 과정에서 각 토큰의 품사를 태깅해놓고, 토큰의 주변을 확인하며 서로 연결되는 토큰들을 그룹핑
- NLTK, spaCy, TextBlob, KoNLPY(한국어) 등의 파이썬 라이버리를 통해 각 토큰을 품사와 맵핑할 수 있음 

In [4]:
import pandas as pd
import json

import spacy

In [5]:
#https://github.com/alicezheng/feature-engineering-book/blob/master/03.02_Chunking_and_POS_Tagging.ipynb 참고

f = open('C:\\Users\\이인주\\Desktop\\2020 summer study\\feature_engineering\\data\\yelp_academic_dataset_review.json', 'r', encoding='UTF-8') #데이터 경로 입력 

js = []
for i in range(10):
    js.append(json.loads(f.readline()))
f.close()

review_df = pd.DataFrame(js)

In [6]:
#https://github.com/alicezheng/feature-engineering-book/blob/master/03.02_Chunking_and_POS_Tagging.ipynb 참고

nlp = spacy.load('en') #en(English) 모델을 불러옴
                      #[E050] Can't find model 'en'. 에러가 뜰 경우 Anaconda Prompt를 관리자 권한으로 실행하여 'python -m spacy download en'를 설치 

In [7]:
#https://github.com/alicezheng/feature-engineering-book/blob/master/03.02_Chunking_and_POS_Tagging.ipynb 참고

doc_df = review_df['text'].apply(nlp) 
doc_df.head()

0    (As, someone, who, has, worked, with, many, mu...
1    (I, am, actually, horrified, this, place, is, ...
2    (I, love, Deagan, 's, ., I, do, ., I, really, ...
3    (Dismal, ,, lukewarm, ,, defrosted, -, tasting...
4    (Oh, happy, day, ,, finally, have, a, Canes, n...
Name: text, dtype: object

In [8]:
#https://github.com/alicezheng/feature-engineering-book/blob/master/03.02_Chunking_and_POS_Tagging.ipynb 참고 

for doc in doc_df[0]:
    print(doc.text, doc.pos_, doc.tag_) #.pos_: simple part-of-speech tag 
                                        #.tag_: detailed part-of-speech tag
                                        #POS 태그 의미: https://dbrang.tistory.com/1245

As SCONJ IN
someone PRON NN
who PRON WP
has AUX VBZ
worked VERB VBN
with ADP IN
many ADJ JJ
museums NOUN NNS
, PUNCT ,
I PRON PRP
was AUX VBD
eager ADJ JJ
to PART TO
visit VERB VB
this DET DT
gallery NOUN NN
on ADP IN
my DET PRP$
most ADV RBS
recent ADJ JJ
trip NOUN NN
to ADP IN
Las PROPN NNP
Vegas PROPN NNP
. PUNCT .
When ADV WRB
I PRON PRP
saw VERB VBD
they PRON PRP
would VERB MD
be AUX VB
showing VERB VBG
infamous ADJ JJ
eggs NOUN NNS
of ADP IN
the DET DT
House PROPN NNP
of ADP IN
Faberge PROPN NNP
from ADP IN
the DET DT
Virginia PROPN NNP
Museum PROPN NNP
of ADP IN
Fine PROPN NNP
Arts PROPN NNP
( PUNCT -LRB-
VMFA PROPN NNP
) PUNCT -RRB-
, PUNCT ,
I PRON PRP
knew VERB VBD
I PRON PRP
had AUX VBD
to PART TO
go VERB VB
! PUNCT .


 SPACE _SP
Tucked VERB VBN
away ADV RB
near SCONJ IN
the DET DT
gelateria PROPN NNP
and CCONJ CC
the DET DT
garden NOUN NN
, PUNCT ,
the DET DT
Gallery PROPN NNP
is AUX VBZ
pretty ADV RB
much ADV RB
hidden VERB VBN
from ADP IN
view NOUN NN
. PUNCT .
It PRON P

In [9]:
#https://github.com/alicezheng/feature-engineering-book/blob/master/03.02_Chunking_and_POS_Tagging.ipynb 참고

print([chunk for chunk in doc_df[0].noun_chunks]) #noun chunking 

[someone, who, many museums, I, this gallery, my most recent trip, Las Vegas, I, they, infamous eggs, the House, Faberge, the Virginia Museum, Fine Arts, VMFA, I, I, the gelateria, the garden, the Gallery, view, It, what, real estate agents, you, wonderful art, a gallery, any size, *s, you, me, you, * pricing, a Las Vegas attraction, the top, the space, the amount, art, you, it, it, them, the security, the show, the curating and design teams, exhibitions, a definite flow, visitors, the art, a certain sequence, it, historical period, cultural significance, audio guides, I, the gallery, I, security, I, such a *fine* institution, I, the lack, knowledge, respect]
